<a href="https://colab.research.google.com/github/kunai-3txk/Compe_tonyobyo/blob/main/AMP_fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [231]:
!pip install catboost
!pip install polars
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.6 MB/s eta 0:00:00


In [170]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import os

import polars as pl

import xgboost as xgb
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_squared_error


import lightgbm as lgb #LightGBM
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor,Pool

# Configuration

In [171]:
class config:
    METRIC = 'RMSE' #RMSE or SMAPE
    RANDOM_STATE=100  
    FRAC = 0.2
    N_FOLD = 5
    
class paths:
    # kaggle環境ならTrue
    if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
        common_path = "/kaggle/input/amp-parkinsons-disease-progression-prediction"
    
    # colaboratory環境ならTrue
    if 'COLAB_GPU' in set(os.environ.keys()):
        common_path = "/content/drive/MyDrive/AMP"
    
    SAMPLE_SUBMISSION = common_path + "/example_test_files/sample_submission.csv"
    SUPPLEMENTAL_CLINICAL_DATA = common_path + "/supplemental_clinical_data.csv"
    TRAIN_CLINICAL_DATA = common_path + "/train_clinical_data.csv"
    TRAIN_PEPTIDES = common_path + "/train_peptides.csv"
    TRAIN_PROTEINS = common_path + "/train_proteins.csv"
    TEST_CLINICAL_DATA = common_path + "/example_test_files/test.csv"
    TEST_PEPTIDES = common_path + "/example_test_files/test_peptides.csv"
    TEST_PROTEINS = common_path + "/example_test_files/test_proteins.csv"
    #train output
    TRAIN = common_path + "/train_all.csv"
    TEST = common_path + "/test_all.csv"
    TE_UPDRS_1 = common_path + "/train_te_updrs_1.csv"
    TE_UPDRS_2 = common_path + "/train_te_updrs_2.csv"
    TE_UPDRS_3 = common_path + "/train_te_updrs_3.csv"
    TE_UPDRS_4 = common_path + "/train_te_updrs_4.csv"
    OUTPUT = common_path

# Function Definition

## make_statistic

In [172]:
#統計量の作成
def make_statistic(df,group_col:str,agg_list:list):
    df_ret = df.groupby(group_col).agg([
        pl.mean(agg_list).suffix('_mean')
        ,pl.std(agg_list).suffix("_std")
        ,pl.median(agg_list).suffix("_med")
        #,pl.col(agg_list).max().suffix("_max") 
        #,pl.col(agg_list).min().suffix("_min") 
        #,pl.quantile(agg_list',0.25).suffix("_quan1")
        #,pl.quantile(agg_list',0.75).suffix("_quan3")
        ])
    return df_ret

## display_feature_importance

In [173]:
def display_feature_importance(feature_list,model):

    # 特徴量重要度を保管する dataframe を用意
    feature_importance = pd.DataFrame()
    feature_importance['feature'] = feature_list
    feature_importance['importance'] = model.feature_importances_

    print('****************************') 
    display(feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False)[:20])

    # order = list(feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False).index)[:30]
    # # 可視化
    # plt.figure(figsize=(10, 10))
    # sns.barplot(x="importance",y="feature",data=feature_importance,order=order)
    # plt.title('importance')
    # plt.tight_layout()
    # plt.show()

## score_cal

In [174]:
def score_cal(y_true, y_pred,metric = config.METRIC):
  if metric == 'SMAPE':
    smap = np.zeros(len(y_true))
    num = np.abs((y_true + 1) - (y_pred+1))
    dem = ((np.abs(y_true + 1) + np.abs(y_pred + 1)) / 2)
    pos_ind = ((y_true+1)!=0)|((y_pred+1)!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    ret = 100 * np.mean(smap)
  if metric == 'RMSE':
    ret = np.sqrt(mean_squared_error(y_true,y_pred))

  return ret

## fit_catboost_fold1

In [175]:
def fit_catboost_fold1(X,y):

  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=config.FRAC, random_state=config.RANDOM_STATE,shuffle=True)

  train_pool = Pool(X_train, y_train)
  model = CatBoostRegressor(random_seed=config.RANDOM_STATE,logging_level='Silent',eval_metric = config.METRIC)
  model.fit(train_pool,plot=False, verbose=False)  

  test_score = score_cal(y_val, model.predict(X_val))
  print('****************************')
  print(f"Test data average score : {test_score:.4f}"," seed ", config.RANDOM_STATE)

  return model,test_score

# Load Data

## train

In [176]:
train_clinical = pd.read_csv(paths.TRAIN_CLINICAL_DATA)
train_peptides = pd.read_csv(paths.TRAIN_PEPTIDES)
train_proteins = pd.read_csv(paths.TRAIN_PROTEINS)
supplemental_clinical = pd.read_csv(paths.SUPPLEMENTAL_CLINICAL_DATA)
#print('Correct label:Proteins shape: (232741, 5) Peptides shape: (981834, 6) Clinical shape: (2615, 8)')
print('Proteins shape:',train_proteins.shape)
print('Peptides shape:',train_peptides.shape)
print('Clinical shape:',train_clinical.shape)
print('supplemental_clinical shape:',supplemental_clinical.shape)

Proteins shape: (232741, 5)
Peptides shape: (981834, 6)
Clinical shape: (2615, 8)
supplemental_clinical shape: (2223, 8)


## test

In [177]:
test_clinical = pd.read_csv(paths.TEST_CLINICAL_DATA)
test_peptides = pd.read_csv(paths.TEST_PEPTIDES)
test_proteins = pd.read_csv(paths.TEST_PROTEINS)
sample_submission= pd.read_csv(paths.SAMPLE_SUBMISSION)
print('Proteins shape:',test_proteins.shape)
print('Peptides shape:',test_peptides.shape)
print('Clinical shape:',test_clinical.shape)

Proteins shape: (453, 6)
Peptides shape: (2057, 7)
Clinical shape: (16, 6)


#preprocessing(common)

In [178]:
train_clinical.replace({'On':1,'Off':0},inplace=True)
supplemental_clinical.replace({'On':1,'Off':0},inplace=True)

#欠損値処理
train_clinical.dropna(subset=['updrs_1'],inplace=True)
train_clinical.dropna(subset=['updrs_2'],inplace=True)
train_clinical.dropna(subset=['updrs_3'],inplace=True) #とりあえず

In [185]:
#お試し。数値情報のlog化

#peptide
train_peptides['PeptideAbundance'] = train_peptides['PeptideAbundance'].apply(np.log)
test_peptides['PeptideAbundance'] = test_peptides['PeptideAbundance'].apply(np.log)
#train_peptides[['PeptideAbundance']].describe()

#proteins
train_proteins['NPX'] = train_proteins['NPX'].apply(np.log)
test_proteins['NPX'] = test_proteins['NPX'].apply(np.log)
#train_proteins[['PeptideAbundance']].describe()


#Common

## make train_sort

In [186]:
#sort用にキー列のみ抽出
train_sort = train_clinical[['visit_id','patient_id','visit_month']]

In [187]:
#test_peptides['visit_id'].unique()

# target encoding

In [188]:
train_clinical_te = train_clinical.copy()
train_peptides_te = train_peptides.copy()
train_proteins_te = train_proteins.copy()
supplemental_te = supplemental_clinical.copy()

## pep,uni <- updrs1-4

In [189]:
#与えられたdfを用いてvisit_idに集約
def pep_te_agg(df_key,df_peptide):
  pep_te_agg = pl.DataFrame(df_peptide[['visit_id','Peptide']]).join(train_pep_te_agg_,on='Peptide',how='left').drop('Peptide').groupby(['visit_id']).agg(
      [
          pl.mean("*").suffix("_mean_by_id"),
          pl.std("*").suffix("_std_by_id"),
      ]
  ).to_pandas()
  pep_te_agg_fill = df_key.merge(pep_te_agg,on='visit_id',how='left').groupby('patient_id').fillna(method='ffill')

  return pep_te_agg_fill

def pro_te_agg(df_key,df_protein):
  uni_te_agg = pl.DataFrame(df_protein[['visit_id','UniProt']]).join(train_uni_te_agg_,on='UniProt',how='left').drop('UniProt').groupby(['visit_id']).agg(
      [
          pl.mean("*").suffix("_mean_by_id"),
          pl.std("*").suffix("_std_by_id"),
      ]
  ).to_pandas()
  pro_te_agg_fill = df_key.merge(uni_te_agg,on='visit_id',how='left').groupby('patient_id').fillna(method='ffill')
  
  return pro_te_agg_fill

In [190]:
train_pep_te = train_peptides_te[['visit_id','Peptide']].merge(train_clinical_te[['visit_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication']]
                                                ,on='visit_id',how='inner')
train_uni_te = train_proteins_te[['visit_id','UniProt']].merge(train_clinical_te[['visit_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication']]
                                                ,on='visit_id',how='inner')

#集計処理 by Pep/Uni
train_pep_te_agg_ = pl.DataFrame(train_pep_te.drop(['visit_id'],axis=1)).groupby(['Peptide']).agg(
    [
        pl.mean("*").suffix("_mean_by_Pep"),
        pl.std("*").suffix("_std_by_Pep"),
        pl.median("*").suffix("_median_by_Pep"),
        pl.max("*").suffix("_max_by_Pep"),
        #pl.min("*").suffix("_min_by_Pep"),
    ]
)

train_uni_te_agg_ = pl.DataFrame(train_uni_te.drop(['visit_id'],axis=1)).groupby(['UniProt']).agg(
    [
        pl.mean("*").suffix("_mean_by_Uni"),
        pl.std("*").suffix("_std_by_Uni"),
        pl.max("*").suffix("_max_by_Uni"),
        #pl.min("*").suffix("_min_by_Uni"),
    ]
)

train_pep_te_agg_fill = pep_te_agg(train_sort,train_peptides_te)
train_pro_te_agg_fill = pro_te_agg(train_sort,train_proteins_te)



## updrs <- updrs

In [191]:
train_clinical_te
supplemental_te

train_clisup_ = pd.concat([train_clinical_te,supplemental_te])
train_clisup = train_clisup_[['updrs_1','updrs_2','updrs_3','updrs_4']]

for i in range(4) :
  train_clisup_tmp = pl.DataFrame(train_clisup).groupby(['updrs_{0}'.format(i+1)]).agg(
    [
        pl.mean("*").suffix("_mean_by_"+'updrs_{0}'.format(i+1)),
        #pl.std("*").suffix("_std_by_Pep"),
        pl.median("*").suffix("_median_by_"+'updrs_{0}'.format(i+1)),
        #pl.max("*").suffix("_max_by_Pep"),
        pl.min("*").suffix("_min_by_"+'updrs_{0}'.format(i+1)),
    ]
  ).to_pandas()
  locals()["te_updrs_{0}".format(i+1)] = train_clisup_tmp

te_updrs_1.to_csv(paths.OUTPUT+'/train_te_updrs_1.csv',index=False)
te_updrs_2.to_csv(paths.OUTPUT+'/train_te_updrs_2.csv',index=False)
te_updrs_3.to_csv(paths.OUTPUT+'/train_te_updrs_3.csv',index=False)
te_updrs_4.to_csv(paths.OUTPUT+'/train_te_updrs_4.csv',index=False)

## month <- updrs1-4

In [192]:
train_clisup = train_clisup_[['visit_month','updrs_1','updrs_2','updrs_3','updrs_4']]

train_te_month_agg = pl.DataFrame(train_clisup).groupby(['visit_month']).agg(
    [
        pl.mean("*").suffix("_mean_by_month"),
        pl.std("*").suffix("_std_by_month"),
        pl.median("*").suffix("_median_by_month"),
        #pl.min("*").suffix("_min_by_Uni"),
    ]
).to_pandas()

#Protain

In [193]:
def feature_proteins(df_key,df_proteins):
  #UniProtのcount情報を付与
  #pro_Uniprot_count = pd.DataFrame(df_proteins['UniProt'].value_counts()).reset_index()
  #df_proteins = df_proteins.merge(pro_Uniprot_count.rename(columns={'UniProt':'value_counts'}),left_on='UniProt',right_on='index',how='left')

  #統計量の作成
  df_proteins_ = pl.DataFrame(df_proteins).select([
      pl.mean('NPX').over('visit_id').alias('NPX_mean_by_id')
    ,pl.std('NPX').over('visit_id').alias('NPX_std_by_id')
    ,pl.median('NPX').over('visit_id').alias('NPX_median_by_id')
    ,pl.mean('NPX').over('UniProt').alias('NPX_mean_by_Uni')
    ,pl.std('NPX').over('UniProt').alias('NPX_std_by_Uni')
    ,pl.median('NPX').over('UniProt').alias('NPX_median_by_Uni')
    #,pl.mean('value_counts').over('visit_id').alias('Unicount_mean_by_id')
    #,pl.std('value_counts').over('visit_id').alias('Unicount_std_by_id')
    #,pl.median('value_counts').over('visit_id').alias('Unicount_median_by_id')
  ])

  df_proteins_agg_ = df_proteins[['visit_id']].join(df_proteins_.to_pandas())
  df_proteins_agg = df_key.merge(df_proteins_agg_.groupby('visit_id').mean().reset_index(),on='visit_id',how='left').sort_values(['patient_id','visit_month']).reset_index(drop=True).drop(['visit_month'],axis=1)

  #欠損値を1つ前の値で埋める
  df_proteins_agg_fill = df_proteins_agg.groupby('patient_id').fillna(method='ffill')
  #df_proteins_toall = df_proteins_agg_fill.merge(train_pro_te_agg_fill,on='visit_id',how='left').copy()
  df_proteins_toall = df_proteins_agg_fill.merge(pro_te_agg(df_key,df_proteins),on='visit_id',how='left').copy()

  return df_proteins_toall

train_proteins_feature = feature_proteins(train_sort,train_proteins)

#Peptide

In [194]:
def feature_peptide(df_key,df_peptides):
  #pep_Peptide_count = pd.DataFrame(df_peptides['Peptide'].value_counts()).reset_index()
  #df_peptides = train_peptides.merge(pep_Peptide_count.rename(columns={'Peptide':'value_counts'}),left_on='Peptide',right_on='index',how='left')

  df_peptides_ = pl.DataFrame(df_peptides).select([
      pl.mean('PeptideAbundance').over('visit_id').alias('Abu_mean_by_id')
    ,pl.std('PeptideAbundance').over('visit_id').alias('Abu_std_by_id')
    ,pl.median('PeptideAbundance').over('visit_id').alias('Abu_median_by_id')
    ,pl.mean('PeptideAbundance').over('Peptide').alias('Abu_mean_by_Pep')
    ,pl.std('PeptideAbundance').over('Peptide').alias('Abu_std_by_Pep')
    ,pl.median('PeptideAbundance').over('Peptide').alias('Abu_median_by_Pep')
    #,pl.mean('value_counts').over('visit_id').alias('Pepcount_mean_by_id')
    #,pl.std('value_counts').over('visit_id').alias('Pepcount_std_by_id')
    #,pl.median('value_counts').over('visit_id').alias('Pepcount_median_by_id')
  ])

  df_peptides_agg_ = df_peptides[['visit_id']].join(df_peptides_.to_pandas())
  df_peptides_agg_  = df_key.merge(df_peptides_agg_.groupby('visit_id').mean().reset_index(),on='visit_id',how='left').sort_values(['patient_id','visit_month']).reset_index(drop=True).drop(['visit_month'],axis=1)

  #欠損値を1つ前の値で埋める
  df_peptides_agg_fill = df_peptides_agg_.groupby('patient_id').fillna(method='ffill')
  df_peptides_agg_toall = df_peptides_agg_fill.merge(pep_te_agg(df_key,df_peptides),on='visit_id',how='left').copy()
  return df_peptides_agg_toall

train_peptides_feature = feature_peptide(train_sort,train_peptides)

#Clinical

In [195]:
#train_clinical['visit_month'].nunique()
#train_clinical['visit_month'].unique()
#train_clinical['visit_month'].value_counts()
#[  0,   3,   6,   9,  12,  18,  24,  30,  36,  42,  48,  54,  60, 72,  84,  96, 108] 17types
#⇒ Phase_1 初期 [0,3,6,9,12,18,24] Phase_2 中期 [30,36,42,48,54,60] Phase_3 後期[72,84,96,108]

In [196]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html

train_clinical_rolling = train_clinical.copy()

for i in range(3):
  train_clinical_rolling['updrs_{0}_rolling'.format(i+1)] = train_clinical_rolling.groupby('patient_id')['updrs_{0}'.format(i+1)].rolling(3,min_periods=1).mean().reset_index(drop=True)
  train_clinical_rolling['updrs_{0}_ewm'.format(i+1)] = train_clinical_rolling.groupby('patient_id')['updrs_{0}'.format(i+1)].transform(lambda x: x.ewm(halflife=10).mean()).reset_index(drop=True)
train_clinical_rolling.head()


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,updrs_1_rolling,updrs_1_ewm,updrs_2_rolling,updrs_2_ewm,updrs_3_rolling,updrs_3_ewm
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,10.000000,10.000000,6.000000,6.000000,15.000000,15.000000
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,10.000000,10.000000,6.500000,6.517322,20.000000,20.173217
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,9.333333,9.286627,7.666667,7.759546,24.666667,25.105042
3,55_9,55,9,8.0,9.0,30.0,0.0,1.0,8.666667,8.930796,8.666667,8.102607,29.666667,26.458798
4,55_12,55,12,10.0,10.0,41.0,0.0,1.0,8.666667,9.175259,9.666667,8.536426,35.000000,29.783493


# supplemental_clinical

In [197]:
supplemental_clinical.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,35_0,35,0,5.0,3.0,16.0,0.0,NaN
1,35_36,35,36,6.0,4.0,20.0,0.0,NaN
2,75_0,75,0,4.0,6.0,26.0,0.0,NaN
3,75_36,75,36,1.0,8.0,38.0,0.0,1.0
4,155_0,155,0,NaN,NaN,0.0,NaN,NaN


#Train All

In [198]:
def make_df_all(df_cli,df_pro,df_pep):
  df_propep = df_pro.merge(df_pep,on='visit_id',how='left')
  df_all = df_cli.merge(df_propep,on='visit_id',how='left')
  return df_all

In [199]:
train_clinical_toall = train_clinical.merge(train_te_month_agg,on='visit_month',how='left')

#train_clinical_toall.shape,train_proteins_feature.shape,train_peptides_feature.shape

train_all = make_df_all(train_clinical_toall,train_proteins_feature,train_peptides_feature)
print('shape',train_all.shape)
train_all.head()

train_all.describe()

train_all.to_csv(paths.OUTPUT+'/train_all.csv',index=False)

train_all.columns
train_proteins_feature.columns
train_peptides_feature.columns


shape (2588, 104)


Index(['visit_id', 'Abu_mean_by_id', 'Abu_std_by_id', 'Abu_median_by_id',
       'Abu_mean_by_Pep', 'Abu_std_by_Pep', 'Abu_median_by_Pep', 'visit_month',
       'updrs_1_mean_by_Pep_mean_by_id', 'updrs_2_mean_by_Pep_mean_by_id',
       'updrs_3_mean_by_Pep_mean_by_id', 'updrs_4_mean_by_Pep_mean_by_id',
       'upd23b_clinical_state_on_medication_mean_by_Pep_mean_by_id',
       'updrs_1_std_by_Pep_mean_by_id', 'updrs_2_std_by_Pep_mean_by_id',
       'updrs_3_std_by_Pep_mean_by_id', 'updrs_4_std_by_Pep_mean_by_id',
       'upd23b_clinical_state_on_medication_std_by_Pep_mean_by_id',
       'updrs_1_median_by_Pep_mean_by_id', 'updrs_2_median_by_Pep_mean_by_id',
       'updrs_3_median_by_Pep_mean_by_id', 'updrs_4_median_by_Pep_mean_by_id',
       'upd23b_clinical_state_on_medication_median_by_Pep_mean_by_id',
       'updrs_1_max_by_Pep_mean_by_id', 'updrs_2_max_by_Pep_mean_by_id',
       'updrs_3_max_by_Pep_mean_by_id', 'updrs_4_max_by_Pep_mean_by_id',
       'upd23b_clinical_state_on_medic

#Test All

3342と50423の6month刻みの予測を行う -> 必要分のみサブ
3342...3342_0は無視。3342_6を3342_0として扱う。50423はそのまま

In [200]:
#clinical
test_clinical.drop(test_clinical.index[test_clinical.visit_id == '3342_6'], inplace=True)
test_clinical.drop(test_clinical.index[test_clinical.visit_id == '50423_6'], inplace=True)
#peptide
test_peptides['visit_id'].replace('3342_6', '3342_0', inplace=True)
test_peptides['visit_month'].replace(6, 0, inplace=True)
#protein
test_proteins['visit_id'].replace('3342_6', '3342_0', inplace=True)
test_proteins['visit_month'].replace(6, 0, inplace=True)


In [201]:
test_sort = test_clinical[['visit_id','visit_month','patient_id']]
test_sort.drop_duplicates(inplace=True)

test_train_peptides = pd.concat([train_peptides,test_peptides.drop(['group_key'],axis=1)],axis=0).reset_index(drop=True)
test_peptide_feature = feature_peptide(test_sort,test_train_peptides)

test_train_proteins = pd.concat([train_proteins,test_proteins.drop(['group_key'],axis=1)],axis=0).reset_index(drop=True)
test_protein_feature = feature_proteins(test_sort,test_train_proteins)

test_clinical_toall = test_sort.merge(train_te_month_agg,on='visit_month',how='left')

#test_all = make_df_all(test_clinical_toall,test_protein_feature,test_peptide_feature)
test_propep = test_protein_feature.merge(test_peptide_feature,on='visit_id',how='left')
test_all = test_clinical_toall.merge(test_propep,on='visit_id',how='left')


#test_all.drop(['updrs_test','row_id','group_key'],axis=1,inplace=True)
test_all.drop_duplicates(inplace=True)

print(test_all.shape)
#test_clinical_toall.shape
#test_protein_feature.shape
#test_peptide_feature.shape

test_all.to_csv(paths.OUTPUT+'/test_all.csv',index=False)


<ipython-input-201-7b8b4d51318f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_sort.drop_duplicates(inplace=True)


(2, 99)


# Fit Start


In [202]:
df_train = train_all.copy()
df_test = test_all.copy()

# Make Test_Val Data

In [203]:
#pivot
df_train_pv=df_train[['visit_id','patient_id','visit_month','updrs_1','updrs_2','updrs_3','updrs_4']].dropna(axis=0).pivot(index="patient_id",columns='visit_month',values='visit_id').reset_index()
df_train_pv.head()

#testデータ用の候補探し
test_patient_candidate_list = list(df_train_pv[
    ~df_train_pv[0].isna() & ~df_train_pv[6].isna() & ~df_train_pv[12].isna() & ~df_train_pv[24].isna()
]['patient_id'].values)
len(test_patient_candidate_list)


56

In [204]:
#test_patient_list = [1517,3863] #決め打ち　Top1,2
test_patient_list = [1517,3863,4923,7265] #決め打ち  Top3,4

test_month_list = [0,6,12,24]   #TESTと同一
#test_month_list = [0,3,6,9,12,18,24]
df_test_val = df_train[df_train['patient_id'].isin(test_patient_list) & df_train['visit_month'].isin(test_month_list)].reset_index(drop=True)

df_test_val_true = pd.DataFrame()

for i in range(0,3):
  df_test_val_true_ = pd.DataFrame()
  df_test_val_true_['id']=df_test_val[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_val_true_['updrs']=df_test_val[['updrs_{0}'.format(i+1)]]
  df_test_val_true = pd.concat([df_test_val_true,df_test_val_true_],axis=0)

# Fit

## fit main

In [ ]:

#predict_order_list = [2,1,3]
predict_order_list = [1,2,3]
#predict_order_list = [3,1,2]
#predict_order_list = [1]

scores = []

df_train_ = df_train.drop(['visit_id','patient_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication'],axis=1)

for order in predict_order_list:
  
  train_X = df_train_.values
  train_y = df_train['updrs_{0}'.format(order)].values
  model,score = fit_catboost_fold1(train_X,train_y)
  scores.append(score)
  if order == 1:
        model_1 = model
        df_te = te_updrs_1
  if order == 2:
        model_2 = model
        df_te = te_updrs_2
  if order == 3:
        model_3 = model
        df_te = te_updrs_3

  #display importance
  display_feature_importance(df_train_.columns,model)

  #予測値をtrainにセット
  df_train_['updrs_{0}'.format(order)] = model.predict(train_X)
  df_train_['updrs_{0}_round'.format(order)] = df_train_['updrs_{0}'.format(order)].round(0)
  #予測値のtarget-encodingテーブルをjoin
  df_te.rename(columns={'updrs_{0}'.format(order):'updrs_{0}_te'.format(order)},inplace=True)
  df_train_ = df_train_.merge(df_te,left_on='updrs_{0}_round'.format(order),right_on='updrs_{0}_te'.format(order),how='left')
  #不要列の削除
  df_train_.drop(['updrs_{0}_round'.format(order),'updrs_{0}_te'.format(order)],axis=1,inplace=True)
  print('train shape:',df_train_.shape)

In [206]:
print('CV....','updrs_1:',scores[0].round(2),'updrs_2:',scores[1].round(2),'updrs_3:',scores[2].round(2))

CV.... updrs_1: 3.67 updrs_2: 3.69 updrs_3: 9.28


###TabNet

In [ ]:
!pip install pytorch-tabnet
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor

In [262]:
EPOCH = 40
BATCH_SIZE = 4

#param
tabnet_params = dict(
    n_d = 8
    ,n_a = 8
    ,n_steps = 3
    ,gamma = 1.3
    ,n_independent = 2
    ,n_shared = 2 
    ,lambda_sparse = 0
    ,optimizer_fn = torch.optim.Adam
    ,optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5)
    ,mask_type = "entmax"
    ,scheduler_params = dict(mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9)
    ,scheduler_fn = torch.optim.lr_scheduler.ReduceLROnPlateau
    ,seed = config.RANDOM_STATE
    ,verbose = 10
)

# null値の数をカウントする
#null_counts = df_train_.isnull().sum()
#null_cols = null_counts[null_counts > 0].index.tolist()
#print(null_cols)

df_train_tn = df_train_.dropna()


train_X = df_train_tn.values
train_y = df_train_tn['updrs_1'].values
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=config.FRAC, random_state=config.RANDOM_STATE,shuffle=True)

model = TabNetRegressor(**tabnet_params)
model.fit(X_train, y_train
          ,eval_set=[(X_val, y_val)]
          ,eval_metric=['mse']
          ,max_epochs=EPOCH
          ,patience=30
          ,num_workers=4 
        )

val_pred = model.predict(X_val)
test_score = score_cal(y_val, model.predict(X_val))
print(test_score)

array([[ 24.        ,   6.30484988,   6.15935335, ...,   0.        ,
          1.        ,   0.        ],
       [ 18.        ,   6.33795014,   6.76177285, ...,   0.        ,
          0.        ,   0.        ],
       [ 12.        ,   5.74579832,   5.76680672, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [ 36.        ,   7.        ,   6.78008299, ...,   0.        ,
          1.        ,   0.        ],
       [108.        ,   9.41666667,   8.16666667, ...,   0.        ,
          3.        ,   0.        ],
       [ 60.        ,   7.26506024,   6.5       , ...,   0.        ,
          0.        ,   0.        ]])

## predict Test_Val

In [218]:
df_test_val_ = df_test_val.drop(['visit_id','patient_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication'],axis=1)

for order in predict_order_list:
  test_X = df_test_val_.values
  test_y = df_test_val['updrs_{0}'.format(order)].values
  if order == 1:
        model_ = model_1
        df_te_val = te_updrs_1
  if order == 2:
        model_ = model_2
        df_te_val = te_updrs_2
  if order == 3:
        model_ = model_3
        df_te_val = te_updrs_3

  df_test_val_['updrs_{0}'.format(order)] = model_.predict(test_X)
  df_test_val_['updrs_{0}_round'.format(order)] = df_test_val_['updrs_{0}'.format(order)].round(0)
  #予測値のtarget-encodingテーブルをjoin
  df_te_val.rename(columns={'updrs_{0}'.format(order):'updrs_{0}_te'.format(order)},inplace=True)
  df_test_val_ = df_test_val_.merge(df_te_val,left_on='updrs_{0}_round'.format(order),right_on='updrs_{0}_te'.format(order),how='left')
  #不要列の削除
  df_test_val_.drop(['updrs_{0}_round'.format(order),'updrs_{0}_te'.format(order)],axis=1,inplace=True)

df_test_val_pred_ = df_test_val[['visit_id']].join(df_test_val_[['updrs_1','updrs_2','updrs_3']])

df_test_val_pred = pd.DataFrame()
for i in range(0,3):
  df_test_val_pred_vs = pd.DataFrame()
  df_test_val_pred_vs['id'] = df_test_val_pred_[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_val_pred_vs['updrs']=df_test_val_pred_[['updrs_{0}'.format(i+1)]]
  df_test_val_pred = pd.concat([df_test_val_pred,df_test_val_pred_vs],axis=0)

print('Score Test Val Predict : ',score_cal(df_test_val_true['updrs'].values,df_test_val_pred['updrs'].values).round(3))
print('Score Test Val Predict SMAPE : ',score_cal(df_test_val_true['updrs'].values,df_test_val_pred['updrs'].values,'SMAPE').round(3))

Score Test Val Predict :  7.223
Score Test Val Predict SMAPE :  25.239


## display Test_Val True vs Pred

In [208]:
#どれぐらい差があるか調べる
df_test_val_truepred = df_test_val_true.merge(df_test_val_pred,on='id',how='left').rename(columns={'updrs_x':'true','updrs_y':'pred'}).round(1)
df_test_val_truepred['diff'] = abs(df_test_val_truepred['true']-df_test_val_truepred['pred'])
df_test_val_truepred['diff_ratio'] = round((df_test_val_truepred['diff'] / df_test_val_truepred['true']) * 100,0)
display(df_test_val_truepred)

,id,true,pred,diff,diff_ratio
0,1517_0_updrs_1,11.0,16.1,5.1,46.0
1,1517_6_updrs_1,17.0,15.3,1.7,10.0
2,1517_12_updrs_1,20.0,18.0,2.0,10.0
3,1517_24_updrs_1,19.0,18.3,0.7,4.0
4,3863_0_updrs_1,8.0,6.3,1.7,21.0
5,3863_6_updrs_1,9.0,7.4,1.6,18.0
6,3863_12_updrs_1,11.0,9.9,1.1,10.0
7,3863_24_updrs_1,11.0,9.7,1.3,12.0
8,4923_0_updrs_1,2.0,6.8,4.8,240.0
9,4923_6_updrs_1,6.0,10.1,4.1,68.0


## predict Test for colab

In [209]:
df_test_ = df_test.drop(['visit_id'],axis=1)

num_features_used = sum(model_1.get_feature_importance() > 0)
print("Number of features used:", num_features_used)

for order in predict_order_list:
  test_X = df_test_.values
  #test_y = df_test['updrs_{0}'.format(order)].values
  if order == 1:
        model_ = model_1
        df_te = te_updrs_1
  if order == 2:
        model_ = model_2
        df_te = te_updrs_2
  if order == 3:
        model_ = model_3
        df_te = te_updrs_3

  df_test_['updrs_{0}'.format(order)] = model_.predict(test_X)
  df_test_['updrs_{0}_round'.format(order)] = df_test_['updrs_{0}'.format(order)].round(0)
  #予測値のtarget-encodingテーブルをjoin
  df_te.rename(columns={'updrs_{0}'.format(order):'updrs_{0}_te'.format(order)},inplace=True)
  df_test_ = df_test_.merge(df_te,left_on='updrs_{0}_round'.format(order),right_on='updrs_{0}_te'.format(order),how='left')
  #不要列の削除
  df_test_.drop(['updrs_{0}_round'.format(order),'updrs_{0}_te'.format(order)],axis=1,inplace=True)

df_test_pred_ = df_test[['visit_id']].reset_index(drop=True).join(df_test_[['updrs_1','updrs_2','updrs_3']]).round(2)

df_test_pred = pd.DataFrame()
for i in range(0,3):
  df_test_pred_vs = pd.DataFrame()
  df_test_pred_vs['id'] = df_test_pred_[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_pred_vs['updrs']=df_test_pred_[['updrs_{0}'.format(i+1)]]
  df_test_pred = pd.concat([df_test_pred,df_test_pred_vs],axis=0)

display(df_test_pred)

Number of features used: 91


,id,updrs
0,3342_0_updrs_1,7.72
1,50423_0_updrs_1,7.58
0,3342_0_updrs_2,6.29
1,50423_0_updrs_2,6.35
0,3342_0_updrs_3,15.05
1,50423_0_updrs_3,15.06


# Inference for kaggle notebook

## import Library

In [210]:
if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
  import amp_pd_peptide
  env = amp_pd_peptide.make_env()
  iter_test = env.iter_test()

In [211]:
# def map_test(x):
#     updrs = x.split('_')[2] + '_' + x.split('_')[3]
#     month = int(x.split('_plus_')[1].split('_')[0])
#     visit_id = x.split('_')[0] + '_' + x.split('_')[1]
#     if updrs=='updrs_4':
#         rating = 0
#     elif updrs =='updrs_1':
#         rating = df[df.visit_id == visit_id]['pred1'].values[0]
#     elif updrs =='updrs_2':
#         rating = df[df.visit_id == visit_id]['pred2'].values[0]
#     else: #updrs =='updrs_3'
#         rating = df[df.visit_id == visit_id]['pred3'].values[0]
#     return rating

if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
  counter = 0

  for (test, test_peptides, test_proteins, sample_submission) in iter_test:

    #1.common 特徴量の作成

    #2-1.predict to updrs_1

    #2-2.updrs_1を使った追加の特徴量作成

    #3-1.predict to updrs_2

    #3-2.updrs_1を使った追加の特徴量作成

    #4.predict to updrs_3
    
    #5.submissionの作成

    sample_submission['rating'] = sample_submission['prediction_id'].apply(map_test)
    env.predict(sample_submission)
    
    if counter == 0:
        display(test)
        display(sample_submission)
        
    counter += 1

In [212]:

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=config.FRAC, random_state=config.RANDOM_STATE,shuffle=True)

# df_log = pd.DataFrame() #Log用DFの定義
# models = []                          # 各 fold のモデル
# scores = 0.0   
# best_score = np.inf
# preds = np.zeros(len(X_test))

# gf = GroupShuffleSplit(n_splits=config.N_FOLD,test_size=0.2,random_state=config.RANDOM_STATE)
# groups = X_train[:,0]

# for fold, (trn_idx, val_idx) in enumerate(gf.split(X_train,y_train,groups)):

#     X_trn, y_trn = X_train[trn_idx,:], y_train[trn_idx]
#     X_val, y_val = X_train[val_idx,:], y_train[val_idx]

#     train_pool = Pool(X_trn, y_trn)
#     validate_pool = Pool(X_val, y_val)

#     model = CatBoostRegressor(random_seed=config.RANDOM_STATE,logging_level='Silent',eval_metric = config.METRIC)
#     model.fit(train_pool, eval_set = validate_pool, early_stopping_rounds = 10, use_best_model=True, plot=False, verbose=False)  
    
#     score = score_cal(y_val,model.predict(X_val))
#     scores += score / config.N_FOLD
#     models.append(model)

#     if score < best_score:
#         best_score = score
#         best_model = model
#         best_fold = fold+1
#     print("Fold :" , fold+1)

# print(f"fold average score : {scores:.4f}")
# print(f"fold best score : {best_score:.4f}"," FOLD ",best_fold)
# #****************************
# # test データ

# # 各モデルで推論
# for model in models:
#     preds += model.predict(X_test) / len(models) 
# test_score = score_cal(y_test, preds)
# pred_best = best_model.predict(X_test)

# print(f"Test data average score : {test_score:.4f}"," seed ", config.RANDOM_STATE)
# print(f"Test data best socre : {score_cal(y_test, pred_best):.4f}"," seed ", config.RANDOM_STATE)
# print("***********************")
# #display_feature_importance(train_column_list,best_model)
# feature_importance = pd.DataFrame()
# feature_importance['feature'] = train_column_list
# feature_importance['importance'] = best_model.feature_importances_

# feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False)[:30]

#TESTのaftermonth予測用。trainから現状とnextの情報を予測する

In [213]:
# 特定の文字列を含む列名を変換する関数
def rename_columns(old_name):
    new_name = old_name.replace('_after', '')
    return new_name

In [214]:
#資材あつめ
train_clinical
train_peptides
train_proteins

print(train_peptides['UniProt'].nunique(),train_peptides['Peptide'].nunique(),train_proteins['UniProt'].nunique())
print(test_peptides['UniProt'].nunique(),test_peptides['Peptide'].nunique(),test_proteins['UniProt'].nunique())
# 227,968,227
# 236,1077,236  -> TESTのほうが、uniとpep多い。無視するしかない


train_clinical_shift = train_clinical.drop(['upd23b_clinical_state_on_medication'],axis=1).join(train_clinical.drop(['visit_id','upd23b_clinical_state_on_medication'],axis=1).groupby('patient_id').shift(-1),rsuffix='_after')
train_clinical_shift.dropna(subset=['visit_month_after'], inplace=True)
train_clinical_shift.describe()


train_clinical_shift_pred = train_clinical_shift[['visit_month','updrs_1','updrs_2','updrs_3','visit_month_after','updrs_1_after','updrs_2_after','updrs_3_after']]

X = train_clinical_shift_pred.drop(['updrs_1_after','updrs_2_after','updrs_3_after'],axis=1).values
y = train_clinical_shift_pred['updrs_1_after'].values
model_u1,score = fit_catboost_fold1(X,y)

X = train_clinical_shift_pred.drop(['updrs_1_after','updrs_2_after','updrs_3_after'],axis=1).values
y = train_clinical_shift_pred['updrs_2_after'].values
model_u2,score = fit_catboost_fold1(X,y)

X = train_clinical_shift_pred.drop(['updrs_1_after','updrs_2_after','updrs_3_after'],axis=1).values
y = train_clinical_shift_pred['updrs_3_after'].values
model_u3,score = fit_catboost_fold1(X,y)

display_feature_importance(train_clinical_shift_pred.drop(['updrs_1_after','updrs_2_after','updrs_3_after'],axis=1).columns,model_u2)


227 968 227
236 1077 236
****************************
Test data average score : 3.2623  seed  100
****************************
Test data average score : 2.9885  seed  100
****************************
Test data average score : 7.5253  seed  100
****************************


feature
updrs_2              58.665397
updrs_3              19.414550
updrs_1              10.647265
visit_month           5.968298
visit_month_after     5.304490
Name: importance, dtype: float64

In [215]:
#EDA
train_clinical_shift_pred[train_clinical_shift_pred['visit_month'] == 0][['updrs_1','updrs_1_after']].groupby('updrs_1').mean().reset_index()
#train_clinical_shift_pred.info()

train_clinical_shift_pred_24 = train_clinical_shift_pred[train_clinical_shift_pred['visit_month'] <= 24]
train_shift_24_pivot=train_clinical_shift_pred_24.pivot_table(index=['visit_month'], aggfunc='mean').reset_index()

from sklearn.linear_model import LinearRegression

coef_intercept_df = pd.DataFrame(columns=['coef', 'intercept'])
for i in range(1, len(train_shift_24_pivot.columns)):
    X = train_shift_24_pivot['visit_month'].values.reshape(-1,1)
    y = train_shift_24_pivot.iloc[:, i].values
    reg = LinearRegression().fit(X, y)
    coef_intercept_df.loc[i] = [reg.coef_[0], reg.intercept_]

coef_intercept_df

,coef,intercept
1,0.053790,5.804065
2,0.059620,6.065409
3,0.015285,5.989678
4,-0.001367,6.465962
5,0.001211,18.099138
6,-0.038773,19.311601
7,1.149165,4.001152


In [216]:
df_test_shift_pred = df_test_pred_.copy()
df_test_shift_pred.drop(['visit_id'],axis=1,inplace=True)
df_test_shift_pred['visit_month']=0
df_test_shift_pred_sum = pd.DataFrame()


for i in range(0,7):
  i_month = i * 6
  df_test_shift_pred['visit_month_after']=df_test_shift_pred['visit_month'] + 6

  df_test_shift_pred['updrs_1_after'] = model_u1.predict(df_test_shift_pred)
  df_test_shift_pred['updrs_2_after'] = model_u2.predict(df_test_shift_pred)
  df_test_shift_pred['updrs_3_after'] = model_u3.predict(df_test_shift_pred)

  df_test_shift_pred_new=df_test_shift_pred[['updrs_1_after','updrs_2_after','updrs_3_after','visit_month_after']]

  df_test_shift_pred_sum = pd.concat([df_test_shift_pred_sum,df_test_shift_pred])

  # 全ての列名にrename_columns関数を適用する
  new_columns = [rename_columns(col_name) for col_name in df_test_shift_pred_new.columns]
  # 新しいdataframeにコピーする
  df_test_shift_pred_new.columns = new_columns
  df_test_shift_pred = df_test_shift_pred_new
  
display(df_test_shift_pred_sum)

<ipython-input-216-283b44c8cedd>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_shift_pred['visit_month_after']=df_test_shift_pred['visit_month'] + 6
<ipython-input-216-283b44c8cedd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_shift_pred['updrs_1_after'] = model_u1.predict(df_test_shift_pred)
<ipython-input-216-283b44c8cedd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,updrs_1,updrs_2,updrs_3,visit_month,visit_month_after,updrs_1_after,updrs_2_after,updrs_3_after
0,7.720000,6.290000,15.050000,0,6,8.877158,13.498781,15.118085
1,7.580000,6.350000,15.060000,0,6,8.877158,13.498781,15.118085
0,8.877158,13.498781,15.118085,6,12,12.826378,11.517320,22.300103
1,8.877158,13.498781,15.118085,6,12,12.826378,11.517320,22.300103
0,12.826378,11.517320,22.300103,12,18,12.747453,17.764526,24.660833
1,12.826378,11.517320,22.300103,12,18,12.747453,17.764526,24.660833
0,12.747453,17.764526,24.660833,18,24,15.869598,20.533716,36.415465
1,12.747453,17.764526,24.660833,18,24,15.869598,20.533716,36.415465
0,15.869598,20.533716,36.415465,24,30,20.775509,22.510692,39.764533
1,15.869598,20.533716,36.415465,24,30,20.775509,22.510692,39.764533
